# Trade Route Creation
## Code reads in an excel file, and parces the three different kinds of trade routes encountered (international, domestic, and incomplete). A count by country (or country combination) for complete (international and domestic) and incomplete routes is then created and exported.
### The output data from this is used in the top species trade route maps.

In [2]:
import pandas as pd

# Read in data
df = pd.read_excel("Data/routes.xlsx")

# Remove whitespace
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Create empty sets and lists
unique_routes = set()
complete_routes = []
incomplete_routes = []

# Iterate through dataframe
for x, row in df.iterrows():
    label = row["label"] 
    species = row["species"]

    # Check if domestic
    domestic = row["route_type"] == "Domestic"
    
    # Extract supply and demand columns
    supply = row["supply country"] if pd.notna(row["supply country"]) else None
    demand = row["demand country"] if pd.notna(row["demand country"]) else None
    
    # Account for up to 3 transit columns
    transit_columns = [f"transit country {i}" for i in range(1, 4)]
    transit = [row[col] for col in transit_columns if col in df.columns and pd.notna(row[col])]
    
    # Domestic route (source data is formatted to contain a supply country in these cases)
    if domestic and supply:
        route = (label, species, supply, supply)
        if route not in unique_routes:
            complete_routes.append((species, supply, supply, 1))
            unique_routes.add(route)
            
    # Non-domestic routes
    else:
        # Process routes pairs in order
        full_route = ([supply] if supply else []) + transit + ([demand] if demand else [])
        
        # Add complete routes (Supply, Transit and Demand)
        for i in range(len(full_route) - 1):
            route = (label, species, full_route[i], full_route[i + 1])
            if route not in unique_routes:
                complete_routes.append((species, full_route[i], full_route[i + 1], 1))
                unique_routes.add(route)
        
        # Transit and Demand
        if not supply and transit:
            for i in range(len(transit) - 1):
                route = (label, species, transit[i], transit[i + 1])
                if route not in unique_routes:
                    complete_routes.append((species, transit[i], transit[i + 1], 1))
                    unique_routes.add(route)
            route = (label, species, transit[-1], demand if demand else None)
            if route not in unique_routes:
                if demand:
                    complete_routes.append((species, transit[-1], demand, 1))
                else:
                    incomplete_routes.append((species, transit[-1], None, label))
                unique_routes.add(route)
                
        # Only Supply
        if supply and not demand and not transit:
            route = (label, species, supply, None)
            if route not in unique_routes:
                incomplete_routes.append((species, supply, None, label))
                unique_routes.add(route)
        
        # Only Transit
        if transit and not supply and not demand:
            for i in range(len(transit) - 1):
                route = (label, species, transit[i], transit[i + 1])
                if route not in unique_routes:
                    complete_routes.append((species, transit[i], transit[i + 1], 1))
                    unique_routes.add(route)
            route = (label, species, transit[-1], None)
            if route not in unique_routes:
                incomplete_routes.append((species, transit[-1], None, label))
                unique_routes.add(route)
                
        # Only Demand
        if demand and not supply and not transit:
            route = (label, species, None, demand)
            if route not in unique_routes:
                incomplete_routes.append((species, None, demand, label))
                unique_routes.add(route)
                
# Export dataframes
complete_routes = pd.DataFrame(complete_routes, columns=["species", "origin", "destination", "count"])
complete_routes = complete_routes.groupby(["species", "origin", "destination"]).sum().reset_index()
complete_routes.to_excel("Output/Complete_Routes.xlsx", index=False)

incomplete_routes = pd.DataFrame(incomplete_routes, columns=["species", "origin", "destination", "label"])
incomplete_routes.to_excel("Output/Incomplete_Routes.xlsx", index=False)

C:\Users\Riley\AppData\Local\Temp\ipykernel_7612\1036809649.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
